In [74]:
import pandas as pd
import numpy as np

In [75]:
hpd = "./purchase_data.csv"

In [76]:
purchase_data = pd.read_csv(hpd)
purchase_data.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [77]:
total_players = purchase_data["SN"].nunique()
total_players

576

In [78]:
uniqueitems = purchase_data['Item ID'].nunique()
avgprice = (purchase_data['Price'].sum()/purchase_data['Price'].count()).round(2)
totalpurchases = purchase_data['Price'].count()
totalrevenue = purchase_data["Price"].sum()

total_analysis_df = pd.DataFrame({"Number of Unique Items": [uniqueitems], 
                              "Average Purchase Price": [avgprice],
                             "Number of Purchases": [totalpurchases],
                             "Total Revenue": [totalrevenue]}, columns= ["Number of Unique Items", "Average Purchase Price",
                            "Number of Purchases", "Total Revenue"])

total_analysis_df.style.format({"Average Purchase Price": "${:.2f}", "Total Revenue": "${:.2f}"})

,Number of Unique Items,Average Purchase Price,Number of Purchases,Total Revenue
0,183,$3.05,780,$2379.77


In [79]:
#malepur = male purchase , maleper = male percentrage etc
allcount = purchase_data["SN"].nunique()
malecount = purchase_data[purchase_data["Gender"] == "Male"]["SN"].nunique()
femalecount = purchase_data[purchase_data["Gender"] == "Female"]["SN"].nunique()
otherpur = totalpurchases - malepur - femalepur
maleper = ((malecount/fullcount)*100)
femaleper = ((femalecount/fullcount)*100)
otherper = ((othercount/fullcount)*100)
malepur = purchase_data[purchase_data["Gender"] == "Male"]["Price"].count()
femalepur = purchase_data[purchase_data["Gender"] == "Female"]["Price"].count()
otherpur = totalpurchases - malepur - femalepur



gender_demo_df = pd.DataFrame({"Gender": ["Male", "Female", "Other / Non-Disclosed"], 
                               "Total Count": [malecount, femalecount, othercount],
                              
                               "Percentage of Players": [maleper, femaleper, otherper]},
                              columns = ["Gender", "Total Count", "Percentage of Players"])
                                        
gender_demo_final = gender_demo_df.set_index("Gender")
gender_demo_final.style.format({ "Percentage of Players" : "{:.2f}"})

,Total Count,Percentage of Players
Gender,,
Male,484,84.03
Female,81,14.06
Other / Non-Disclosed,11,1.91


In [80]:
#malepriceavg = male Average Purchase Price , malepurt = male Total Purchase Value , maleavgtv = Avg Total Purchase per Person etc
malepriceavg = purchase_data[purchase_data["Gender"] == "Male"]['Price'].mean()
femalepriceavg = purchase_data[purchase_data["Gender"] == "Female"]['Price'].mean()
otherpriceavg = purchase_data[purchase_data["Gender"] == "Other / Non-Disclosed"]['Price'].mean()
malepurt = purchase_data[purchase_data["Gender"] == "Male"]['Price'].sum()
femalepurt = purchase_data[purchase_data["Gender"] == "Female"]['Price'].sum()
otherpurt =purchase_data[purchase_data["Gender"] == "Other / Non-Disclosed"]['Price'].sum()
maleavgtv = malepricet/malecount
femaleavgtv = femalepricet/femalecount
otheravgtv = otherpricet/othercount

gender_purchase_df = pd.DataFrame({"Gender": [ "Female", "Male",  "Other / Non-Disclosed"], 
                                   "Purchase Count": [femalepur, malepur,  otherpur],
                                        "Average Purchase Price": [femalepriceavg, malepriceavg, otherpriceavg],
                                   "Total Purchase Value": [femalepurt, malepurt, otherpurt],
                                "Avg Total Purchase per Person": [femaleavgtv, maleavgtv, otheravgtv]}, 
                                  columns = 
                                        ["Gender", "Purchase Count", 
                                         "Average Purchase Price", "Total Purchase Value", "Avg Total Purchase per Person"])
                                        
gender_purchase_final = gender_purchase_df.set_index("Gender")
gender_purchase_final.style.format({"Average Purchase Price": "${:.2f}",
                                    "Total Purchase Value": "${:.2f}", "Avg Total Purchase per Person" : "${:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,$1967.64,$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [81]:
purchase_data["Age"].min()
purchase_data["Age"].max()
bins=[0,9,14,19,24,29,34,39,45]
group_labels=["<10","10-14","15-19","20-24","25-29","30-34","35-39","40+"]
purchase_data["Age Category"] = pd.cut(purchase_data["Age"], bins, labels=group_labels)
purchase_data.head()
age_group = purchase_data.groupby("Age Category")
len(age_group)
age_group.head()
age_list=[]
age_cnt = []
age_pct = []
purchase_data = purchase_data.sort_values("Age Category")
purchase_data.head()
for age in purchase_data["Age Category"].unique():
    age_list.append(age)
    age_cnt.append(age_group.get_group(age)["SN"].nunique())
    age_pct.append((age_group.get_group(age)["SN"].nunique()/total_players)*100 )
age_demo = pd.DataFrame({"Age":age_list,"Total Counts":age_cnt,"Percentage of Players":age_pct})
age_demo["Percentage of Players"] = age_demo["Percentage of Players"].astype(float).map("{:.02f}".format)
age_demo = age_demo.sort_values("Total Counts" , ascending=False)
age_demo.sort_index(inplace=True)
age_demo.set_index("Age", drop=True, inplace=True)
age_demo.index.name=None
age_demo.head(100)

,Total Counts,Percentage of Players
<10,17,2.95
10-14,22,3.82
15-19,107,18.58
20-24,258,44.79
25-29,77,13.37
30-34,52,9.03
35-39,31,5.38
40+,12,2.08


In [82]:
purchase_analysis_age = age_group.agg({"Price":["count","mean","sum"]})
purchase_analysis_age.rename(columns={ "count":"Purchase Count",
                                  "mean":"Average Purchase Price",
                                  "sum":"Total Purchase Value"
                                 },
                        inplace=True)

purchase_analysis_age.columns = ["Purchase Count" , "Average Purchase Price" , 
                             "Total Purchase Value" 
                            ]
purchase_analysis_age["Avg Total Purchase per Person"] =purchase_analysis_age["Total Purchase Value"] / age_demo["Total Counts"]
purchase_analysis_age["Average Purchase Price"]= purchase_analysis_age["Average Purchase Price"].astype("float").map("${:.02f}".format)
purchase_analysis_age["Total Purchase Value"] = purchase_analysis_age["Total Purchase Value"].astype("float").map("${:,.02f}".format)
purchase_analysis_age["Avg Total Purchase per Person"] = purchase_analysis_age["Avg Total Purchase per Person"].astype("float").map("${:.02f}".format)
purchase_analysis_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Age Category,,,,
<10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-39,41,$3.60,$147.67,$4.76
40+,13,$2.94,$38.24,$3.19


In [83]:
sn_total_purchase = purchase_data.groupby('SN')['Price'].sum().to_frame()
sn_purchase_count = purchase_data.groupby('SN')['Price'].count().to_frame()
sn_purchase_avg = purchase_data.groupby('SN')['Price'].mean().to_frame()

sn_total_purchase.columns=["Total Purchase Value"]
join_one = sn_total_purchase.join(sn_purchase_count, how="left")
join_one.columns=["Total Purchase Value", "Purchase Count"]

join_two = join_one.join(sn_purchase_avg, how="inner")
join_two.columns=["Total Purchase Value", "Purchase Count", "Average Purchase Price"]

top_spenders_df = join_two[["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]
top_spenders_final = top_spenders_df.sort_values('Total Purchase Value', ascending=False).head()
top_spenders_final.style.format({"Average Purchase Price": "${:.2f}", "Total Purchase Value": "${:.2f}"})

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.41,$13.62
Iskadarya95,3,$4.37,$13.10


In [84]:
popular_group = purchase_data.groupby(by=["Item ID","Item Name"])
popular_df = popular_group.agg({"Price":["count","sum"]})
popular_df.rename(columns={"count":"Purchase Count","Price":"Price","sum":"Total Purchase Value"},inplace=True)
popular_df.columns=popular_df.columns.droplevel(level=0)
item_price_df = purchase_data[["Item ID","Item Name","Price"]]
item_price_df = item_price_df.drop_duplicates(subset=("Item ID","Item Name","Price"))
popular_df.reset_index(inplace = True)
popular_df = popular_df.merge(item_price_df,on="Item ID", how="inner")[["Item ID","Item Name_x","Purchase Count","Price","Total Purchase Value"]]
popular_df.rename(columns={"Item Name_x":"Item Name"},inplace=True)
popular_df.set_index(keys=["Item ID","Item Name"],inplace=True)
popular_df.sort_values("Purchase Count",inplace=True,ascending=False)
popular_df["Total Purchase Value"] = popular_df["Total Purchase Value"].astype("float").map("${:.02f}".format)
popular_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
145,Fiery Glass Crusader,9,4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,3.53,$31.77
82,Nirvana,9,4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,1.02,$8.16


In [85]:
popular_df["Total Purchase Value"] = popular_df["Total Purchase Value"].str.replace("$","").astype("float")
popular_df = popular_df.sort_values("Total Purchase Value",ascending=False)
popular_df["Total Purchase Value"] = popular_df["Total Purchase Value"].astype("float").map("${:.02f}".format)
popular_df.head()
profitable_df = popular_df
profitable_df.head()

,,Purchase Count,Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,4.23,$50.76
82,Nirvana,9,4.90,$44.10
145,Fiery Glass Crusader,9,4.58,$41.22
92,Final Critic,8,4.88,$39.04
103,Singed Scalpel,8,4.35,$34.80
